### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/319_TROCAS7_Fusion_Apr2021_PEAKS_186/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/319_TROCAS7_Fusion_Apr2021_PEAKS_186


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 319_TROCAS7_Fusion_Apr2021_PEAKS_186_peptide.csv
 319_TROCAS7_Fusion_Apr2021_PEAKS_186_protein.csv
 319_TROCAS7_Fusion_Apr2021_PEAKS_186_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup318 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/319_TROCAS7_Fusion_Apr2021_PEAKS_186/319_TROCAS7_Fusion_Apr2021_PEAKS_186_peptide.csv")

# remove redundant rows
peaksdb318 = pd.DataFrame.drop_duplicates(peaksdbdup318)

print(peaksdb318.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb318.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb318['#Spec']
del peaksdb318['Accession']
del peaksdb318['PTM']
del peaksdb318['AScore']

mean_length = peaksdb318['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup318))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb318))

#look at the dataframe
peaksdb318.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-319-NMCP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-319-NMCP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.32857142857143
# redundant peaksdb peptides in combined dataframe 140
# nonredundant peaksdb peptides in combined dataframe 140


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,101.27,1881.8350,18,2.5,941.9271,93.10,4780000.0,113,16129,20210411_Trocas7_669_NMCP319_DDA_120min_1.raw
1,LGEHNIDVLEGNEQFINAAK,95.42,2210.0967,20,1.9,737.7076,96.33,155000.0,113,16854,20210411_Trocas7_669_NMCP319_DDA_120min_1.raw
2,SGGGGGGGLGSGGSIR,85.86,1231.5905,16,1.9,616.8036,39.30,270000.0,113,6581,20210411_Trocas7_669_NMCP319_DDA_120min_1.raw
3,SSGSSYPSLLQC(+57.02)LK,83.08,1525.7446,14,1.9,763.8810,101.79,570000.0,113,18151,20210411_Trocas7_669_NMCP319_DDA_120min_1.raw
4,SGGGGGGGGC(+57.02)GGGGGVSSLR,70.90,1548.6699,20,2.5,775.3442,38.99,28800.0,113,6526,20210411_Trocas7_669_NMCP319_DDA_120min_1.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb318['A'] = peaksdb318['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb318['C'] = peaksdb318['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb318['D'] = peaksdb318['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb318['E'] = peaksdb318['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb318['F'] = peaksdb318['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb318['G'] = peaksdb318['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb318['H'] = peaksdb318['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb318 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb318['I'] = peaksdb318['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb318['K'] = peaksdb318['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb318['L'] = peaksdb318['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb318['M'] = peaksdb318['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb318['N'] = peaksdb318['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb318['P'] = peaksdb318['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb318['Q'] = peaksdb318['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb318['R'] = peaksdb318['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb318['S'] = peaksdb318['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb318['T'] = peaksdb318['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb318['V'] = peaksdb318['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb318['W'] = peaksdb318['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb318['Y'] = peaksdb318['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb318['c-carb'] = peaksdb318['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb318['m-oxid'] = peaksdb318['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb318['n-deam'] = peaksdb318['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb318['q-deam'] = peaksdb318['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb318['stripped_peptide'] = peaksdb318['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb318['stripped_length'] = peaksdb318['stripped_peptide'].apply(len)

peaksdb318['NAAF_num.'] = peaksdb318['Area'] / peaksdb318['stripped_length']

# total the number of modifications in sequence
peaksdb318['ptm-total'] = peaksdb318['c-carb'] + peaksdb318['m-oxid'] + peaksdb318['n-deam'] + peaksdb318['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb318['stripped_IL']= peaksdb318['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB.csv")

# check out the results
peaksdb318.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,101.27,1881.8350,18,2.5,941.9271,93.10,4780000.0,113,16129,...,0,2,0,0,0,SCLISGWGNTK,11,434545.454545,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,95.42,2210.0967,20,1.9,737.7076,96.33,155000.0,113,16854,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,7750.000000,0,LGEHNLDVLEGNEQFLNAAK
2,SGGGGGGGLGSGGSIR,85.86,1231.5905,16,1.9,616.8036,39.30,270000.0,113,6581,...,0,0,0,0,0,SGGGGGGGLGSGGSIR,16,16875.000000,0,SGGGGGGGLGSGGSLR
3,SSGSSYPSLLQC(+57.02)LK,83.08,1525.7446,14,1.9,763.8810,101.79,570000.0,113,18151,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,40714.285714,1,SSGSSYPSLLQCLK
4,SGGGGGGGGC(+57.02)GGGGGVSSLR,70.90,1548.6699,20,2.5,775.3442,38.99,28800.0,113,6526,...,0,1,0,0,0,SGGGGGGGGCGGGGGVSSLR,20,1440.000000,1,SGGGGGGGGCGGGGGVSSLR


In [6]:
# keep only stripped peptide I/L and NAAF
db_318 = peaksdb318[['stripped_IL', 'Area', 'NAAF_num.']]

db_318.set_index('stripped_IL')

# write modified dataframe to new txt file
db_318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/318C_MN_T24_GF_PDB_ILnaafs.csv")

db_318.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,4780000.0,434545.454545
1,LGEHNLDVLEGNEQFLNAAK,155000.0,7750.000000
2,SGGGGGGGLGSGGSLR,270000.0,16875.000000
3,SSGSSYPSLLQCLK,570000.0,40714.285714
4,SGGGGGGGGCGGGGGVSSLR,28800.0,1440.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep318moddup = peaksdb318[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep318dup = peaksdb318[["stripped_peptide"]]

# deduplicate both of these lists
pep318mod = pep318moddup.drop_duplicates()
pep318 = pep318dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 318:", len(pep318moddup))
print("Deduplicated modified PeaksDB peptides in 318:", len(pep318mod))
print("Total stripped PeaksDB peptides in 318:", len(pep318dup))
print("Deduplicated stripped PeaksDB peptides in 318:", len(pep318))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB_stripped_peptides.fas

# look at the stripped peptides
pep318.head()

Total modified PeaksDB peptides in 318: 140
Deduplicated modified PeaksDB peptides in 318: 140
Total stripped PeaksDB peptides in 318: 140
Deduplicated stripped PeaksDB peptides in 318: 132


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
2,SGGGGGGGLGSGGSIR
3,SSGSSYPSLLQCLK
4,SGGGGGGGGCGGGGGVSSLR


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb318['A'].sum(),
        'C': peaksdb318['C'].sum(),
        'D': peaksdb318['D'].sum(),
        'E': peaksdb318['E'].sum(),
        'F': peaksdb318['F'].sum(),
        'G': peaksdb318['G'].sum(),
        'H': peaksdb318['H'].sum(),
        'I': peaksdb318['I'].sum(),
        'K': peaksdb318['K'].sum(),
        'L': peaksdb318['L'].sum(),
        'M': peaksdb318['M'].sum(),
        'N': peaksdb318['N'].sum(),
        'P': peaksdb318['P'].sum(),
        'Q': peaksdb318['Q'].sum(),
        'R': peaksdb318['R'].sum(),
        'S': peaksdb318['S'].sum(),
        'T': peaksdb318['T'].sum(),
        'V': peaksdb318['V'].sum(),
        'W': peaksdb318['W'].sum(),
        'Y': peaksdb318['Y'].sum(),
        'c-carb': peaksdb318['c-carb'].sum(),
        'm-oxid': peaksdb318['m-oxid'].sum(),
        'n-deam': peaksdb318['n-deam'].sum(),
        'q-deam': peaksdb318['q-deam'].sum(),
        'Total area': peaksdb318['Area'].sum(),
        'Total length': peaksdb318['stripped_length'].sum()
       }

totalpeaksdb318 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb318['% C w/ carb'] = totalpeaksdb318['c-carb'] / totalpeaksdb318['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb318['% M w/ oxid'] = totalpeaksdb318['m-oxid'] / totalpeaksdb318['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb318['% N w/ deam'] = totalpeaksdb318['n-deam'] / totalpeaksdb318['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb318['% Q w/ deam'] = totalpeaksdb318['q-deam'] / totalpeaksdb318['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb318['NAAF denom.'] = totalpeaksdb318['Total area'] / totalpeaksdb318['Total length']

# write modified dataframe to new txt file
totalpeaksdb318.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB_totals.csv")

totalpeaksdb318.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,115,19,38,58,54,220,29,124,112,182,...,5,13,11,342702470.5,1527,1.0,0.454545,0.178082,0.275,224428.598887


In [9]:
# use the calculated NAAF factor (in totalpeaksdb318 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 224428.598887

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb318['NAAF factor'] = (peaksdb318['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb318_NAAF = peaksdb318[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb318_NAAF['A-NAAF20'] = peaksdb318_NAAF['A'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['C-NAAF20'] = peaksdb318_NAAF['C'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['D-NAAF20'] = peaksdb318_NAAF['D'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['E-NAAF20'] = peaksdb318_NAAF['E'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['F-NAAF20'] = peaksdb318_NAAF['F'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['G-NAAF20'] = peaksdb318_NAAF['G'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['H-NAAF20'] = peaksdb318_NAAF['H'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['I-NAAF20'] = peaksdb318_NAAF['I'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['K-NAAF20'] = peaksdb318_NAAF['K'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['L-NAAF20'] = peaksdb318_NAAF['L'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['M-NAAF20'] = peaksdb318_NAAF['M'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['N-NAAF20'] = peaksdb318_NAAF['N'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['P-NAAF20'] = peaksdb318_NAAF['P'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['Q-NAAF20'] = peaksdb318_NAAF['Q'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['R-NAAF20'] = peaksdb318_NAAF['R'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['S-NAAF20'] = peaksdb318_NAAF['S'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['T-NAAF20'] = peaksdb318_NAAF['T'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['V-NAAF20'] = peaksdb318_NAAF['V'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['W-NAAF20'] = peaksdb318_NAAF['W'] * peaksdb318['NAAF factor']
peaksdb318_NAAF['Y-NAAF20'] = peaksdb318_NAAF['Y'] * peaksdb318['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb318_NAAF['ccarb-NAAF20'] = peaksdb318_NAAF['c-carb'] * peaksdb318_NAAF['NAAF factor']
peaksdb318_NAAF['moxid-NAAF20'] = peaksdb318_NAAF['m-oxid'] * peaksdb318_NAAF['NAAF factor']
peaksdb318_NAAF['ndeam-NAAF20'] = peaksdb318_NAAF['n-deam'] * peaksdb318_NAAF['NAAF factor']
peaksdb318_NAAF['qdeam-NAAF20'] = peaksdb318_NAAF['q-deam'] * peaksdb318_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb318_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB_naaf.csv")

peaksdb318_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,1.936230,3,2,0,1,0,3,0,1,...,0.000000,3.872461,3.872461,0.000000,1.93623,0.000000,3.872461,0.0,0.0,0.0
1,LGEHNIDVLEGNEQFINAAK,0.034532,2,0,1,3,1,2,1,2,...,0.000000,0.000000,0.000000,0.034532,0.00000,0.000000,0.000000,0.0,0.0,0.0
2,SGGGGGGGLGSGGSIR,0.075191,0,0,0,0,0,10,0,1,...,0.075191,0.225573,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0
3,SSGSSYPSLLQCLK,0.181413,0,1,0,0,0,1,0,0,...,0.000000,0.907065,0.000000,0.000000,0.00000,0.181413,0.181413,0.0,0.0,0.0
4,SGGGGGGGGCGGGGGVSSLR,0.006416,0,1,0,0,0,13,0,0,...,0.006416,0.019249,0.000000,0.006416,0.00000,0.000000,0.006416,0.0,0.0,0.0


In [10]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb318 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb318_NAAF['NAAF factor'].sum(),
        'A': peaksdb318_NAAF['A-NAAF20'].sum(),
        'C': peaksdb318_NAAF['C-NAAF20'].sum(),
        'D': peaksdb318_NAAF['D-NAAF20'].sum(),
        'E': peaksdb318_NAAF['E-NAAF20'].sum(),
        'F': peaksdb318_NAAF['F-NAAF20'].sum(),
        'G': peaksdb318_NAAF['G-NAAF20'].sum(),
        'H': peaksdb318_NAAF['H-NAAF20'].sum(),
        'I': peaksdb318_NAAF['I-NAAF20'].sum(),
        'K': peaksdb318_NAAF['K-NAAF20'].sum(),
        'L': peaksdb318_NAAF['L-NAAF20'].sum(),
        'M': peaksdb318_NAAF['M-NAAF20'].sum(),
        'N': peaksdb318_NAAF['N-NAAF20'].sum(),
        'P': peaksdb318_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb318_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb318_NAAF['R-NAAF20'].sum(),
        'S': peaksdb318_NAAF['S-NAAF20'].sum(),
        'T': peaksdb318_NAAF['T-NAAF20'].sum(),
        'V': peaksdb318_NAAF['V-NAAF20'].sum(),
        'W': peaksdb318_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb318_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb318_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb318_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb318_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb318_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb318_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb318_NAAF['% C w/ carb.'] = totalpeaksdb318_NAAF['c-carb'] / totalpeaksdb318_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb318_NAAF['% M w/ oxid'] = totalpeaksdb318_NAAF['m-oxid'] / totalpeaksdb318_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb318_NAAF['% N w/ deam'] = totalpeaksdb318_NAAF['n-deam'] / totalpeaksdb318_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb318_NAAF['% Q w/ deam'] = totalpeaksdb318_NAAF['q-deam'] / totalpeaksdb318_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb318_NAAF['NAAF check'] = totalpeaksdb318_NAAF['NAAF'] / 224428.598887

# write modified dataframe to new txt file, same name + totals
totalpeaksdb318_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/318_MN_T24_GF_PDB/318C_MN_T24_GF_PDB_naaf_totals.csv")

totalpeaksdb318_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,180.188989,115.722463,5.64937,5.186264,7.981363,2.305823,22.664967,4.713267,10.046485,12.372276,...,1.820971,5.64937,0.053591,0.64698,0.510417,1.0,0.016723,0.018449,0.060695,0.000803
